In [1]:
####

In [2]:
import gym
import collections

In [3]:
class Config:
    def __init__(self):
        self.GAMMA = 0.9
        self.ENV_NAME = 'FrozenLake-v0'
        self.TEST_EPISODES = 20

In [43]:
class Agent:
    def __init__(self , 
                 Config = Config):
        
        self.config = Config()
        self.env = gym.make(self.config.ENV_NAME)
        
        self.reward = collections.defaultdict(float)
        self.transists = collections.defaultdict(collections.Counter)
        self.q_values = collections.defaultdict(float)
        self.state = self.env.reset()

    def play_n_random_steps(self , steps):
        for _ in range(steps):
            action = self.env.action_space.sample()
            new_state , reward , is_done , _ = self.env.step(action)
            
            self.reward[(self.state , action , new_state)] = reward
            self.transists[(self.state , action)][new_state] += 1
            
            self.state = self.env.reset() if is_done else new_state

    def select_action(self , state):
        best_action , best_value = None , None

        for action in range(self.env.action_space.n):
            action_value = self.q_values[(state , action)]
            if best_action is None or best_value < action_value:
                best_action = action
                best_value = action_value
        return best_action

    def q_value_iteration(self):
        for state in range(self.env.observation_space.n):
            for action in range(self.env.action_space.n):
                action_values = 0.0
                target_counts = self.transists[(state , action)]
                total = sum(target_counts.values())

                for tgt_state , count in target_counts.items():
                    key = (state , action , tgt_state)
                    reward = self.reward[key]
                    best_action = self.select_action(tgt_state)
                    val = reward + self.config.GAMMA * self.q_values[(tgt_state , best_action)]
                    action_values += (count / total) * val
                self.q_values[(state , action)] = action_values

    def play_episode(self , env):
        total_reward = 0.0
        state = env.reset()

        while True:
            action = self.select_action(state)
            new_state , reward , is_done , _ = env.step(action)
            
            self.reward[(state , action , new_state)] = reward
            self.transists[(state , action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

In [44]:
config = Config()
agent = Agent()
env = gym.make(config.ENV_NAME)

In [ ]:
iter_no = 0
best_reward = 0.0

while True:
    env.render()
    iter_no += 1

    agent.play_n_random_steps(100)
    agent.q_value_iteration()
    reward = 0.0

    for _ in range(config.TEST_EPISODES):
        reward += agent.play_episode(env)
    reward /= config.TEST_EPISODES

    if reward > best_reward:
        print(f'Best reward updated {best_reward} , {reward}')
        best_reward = reward
    if reward > 0.80:
        print(f'Solved in iterations {iter_no}')
        break